In [5]:
from fastapi import FastAPI
import pandas as pd
#import pickle

In [6]:
app = FastAPI()
# http://127.0.0.1:8000 

In [125]:
# Importamos dataset
ruta_movies_df = r"csv\dmovies_df_limpio.csv"
movies_df = pd.read_csv(ruta_movies_df)
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'], format='%Y-%m-%d', errors='coerce')

ruta_credits_df = r"csv\dcredits_limpio.csv"
credits_df = pd.read_csv(ruta_credits_df)

#Df = pd.read_csv(r'Df_recomendacion.csv')

#with open('similarity_matrix.pickle', 'rb') as f:
 #   cosine_sim = pickle.load(f)


C:\Users\Daumian Ruiz\AppData\Local\Temp\ipykernel_3616\4208819685.py:3: DtypeWarning: Columns (5,10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv(ruta_movies_df)


In [61]:
#def cantidad_filmaciones_mes( Mes ): Se ingresa un mes en idioma Español. 
# Debe devolver la cantidad de películas que 
# fueron estrenadas en el mes consultado en la totalidad del dataset.



def peliculas_mes(mes:str):
   meses = {'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4, 'mayo': 5, 'junio': 6,'julio': 7, 'agosto': 8, 'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12}
   mes_numerico = meses.get(mes.lower())


   #Prevencion de errores
   if mes_numerico is None:
      return "error: Dato erroneo, por favor envia un dato en formato texto, por ejemplo 'febrero'" 

   peliculas_en_este_mes = movies_df[movies_df['release_date'].dt.month == mes_numerico]
   num_peliculas = len(peliculas_en_este_mes)
   #return
   return {f'el mes de {mes} se extrenaron {num_peliculas} peliculas'}  
   

In [ ]:
def buscar_titulo(texto, df):
    resultados = []
    for index, row in df.iterrows():
        titulo = row['title']
        fecha_lanzamiento = row['release_date']
        calificacion = row['vote_average']
        
        if texto.lower() in titulo.lower():
            resultado = f"Título: {titulo}\nFecha de lanzamiento: {fecha_lanzamiento}\nCalificación: {calificacion}\n"
            resultados.append(resultado)
    
    return '\n'.join(resultados)


In [63]:
@app.get('/cantidad_filmaciones_mes/{mes}')
async def cantidad_filmaciones_mes(mes: str):
   can_peliculas=peliculas_mes(mes)
   return can_peliculas

In [65]:
def peliculas_dia(dia:int):
   dia_numerico = dia

   #Prevencion de errores
   if dia_numerico is None:
      return "error: Dato erroneo, por favor envia un dato en formato numero, desde el 1 al 31" 

   peliculas_en_este_dia = movies_df[movies_df['release_date'].dt.day == dia_numerico]
   num_peliculas = len(peliculas_en_este_dia)
   #return
   return {f'el dia {dia} del mes, se extrenaron {num_peliculas} peliculas'}  

In [107]:
def pelicula_datos(titulo:str):
    coincidencias = movies_df[movies_df['title'].str.contains(titulo, case=False)]
    resultados = coincidencias[['title', 'release_year', 'vote_average']].values.tolist()
    resultados_formateados = '\n'.join([f"Título: {r[0]} | Fecha de lanzamiento: {r[1]} | Calificación: {r[2]}" for r in resultados])
    return resultados_formateados

In [112]:
texto_buscado = "nines"  
resultados = pelicula_datos(texto_buscado)

print(resultados)

Título: The Nines | Fecha de lanzamiento: 2007.0 | Calificación: 6.1


In [118]:
def votos_titulo(titulo:str):
    coincidencias = movies_df[(movies_df['title'].str.contains(titulo, case=False)) & (movies_df['vote_count'] > 2000)]
    resultados = coincidencias[['title', 'vote_average', 'vote_count']].values.tolist()
    resultados_formateados = '\n'.join([f"Título: {r[0]} | Puntuacion: {r[1]} | Cantidad de votos: {r[2]}" for r in resultados])
    return resultados_formateados

In [122]:
texto_buscado = "ke"  
resultados = votos_titulo(texto_buscado)

print(resultados)

Título: Twelve Monkeys | Puntuacion: 7.4 | Cantidad de votos: 2470.0
Título: The Empire Strikes Back | Puntuacion: 8.2 | Cantidad de votos: 5998.0
Título: Full Metal Jacket | Puntuacion: 7.9 | Cantidad de votos: 2595.0
Título: Princess Mononoke | Puntuacion: 8.2 | Cantidad de votos: 2041.0
Título: Taken | Puntuacion: 7.2 | Cantidad de votos: 4444.0
Título: Taken 2 | Puntuacion: 6.1 | Cantidad de votos: 2783.0
Título: Taken 3 | Puntuacion: 6.1 | Cantidad de votos: 2225.0
Título: Star Wars: The Force Awakens | Puntuacion: 7.5 | Cantidad de votos: 7993.0
Título: Fifty Shades Darker | Puntuacion: 6.0 | Cantidad de votos: 2341.0


In [130]:
def obtener_informacion_actor(actor_nombre):
    coincidencias = credits_df[credits_df['cast'].str.contains(actor_nombre, case=False)]
    ids_peliculas = coincidencias['id'].tolist()
    peliculas_participadas = movies_df[movies_df['id'].isin(ids_peliculas)]
    num_peliculas = len(peliculas_participadas)
    suma_recaudacion = peliculas_participadas['return_of_invertion'].sum()
    
    return actor_nombre, num_peliculas, suma_recaudacion


In [145]:
actor_buscado = "Liam N"
resultado = obtener_informacion_actor(actor_buscado)
nombre_actor, num_peliculas, recaucacion = resultado
print(f"{nombre_actor} participó en {num_peliculas} películas y recaudó un total de {recaucacion}.")

Liam N participó en 126 películas y recaudó un total de 184.43293214931893.


In [215]:
def get_director(director_nombre):

    coincidencias = credits_df[credits_df['crew'].str.contains(director_nombre, case=False)]   
    ids_peliculas = coincidencias['id'].tolist()
    peliculas_dirigidas = movies_df[movies_df['id'].isin(ids_peliculas)]
    datos_mostrar = peliculas_dirigidas[['title', 'release_year','revenue','budget','return_of_invertion']].values.tolist()
    resultados_sumados = '\n'.join([f"Título: {r[0]} | Lanzamiento: {r[1]} | Recaudacion: {r[2]} | Presupuesto: {r[3]} |Return of Invertion: {r[4]} | " for r in datos_mostrar])
    
    suma_recaudacion = peliculas_dirigidas['return_of_invertion'].sum()
    Titulo = f'{director_nombre} ha recaudado un total de {suma_recaudacion} dirigiendo :'
    final_director = Titulo + "\n" + resultados_sumados

    return final_director

In [216]:
director_buscado = "Mel G"  # Nombre del actor que deseas buscar
resultados = get_director(director_buscado)  # credits_df y movies_df son tus DataFrames
print(resultados)

Mel G ha recaudado un total de 35.59412502777778 dirigiendo :
Título: Braveheart | Lanzamiento: 1995.0 | Recaudacion: 210000000.0 | Presupuesto: 72000000 |Return of Invertion: 2.9166666666666665 | 
Título: The Man without a Face | Lanzamiento: 1993.0 | Recaudacion: 24760338.0 | Presupuesto: 0 |Return of Invertion: 0.0 | 
Título: One Eight Seven | Lanzamiento: 1997.0 | Recaudacion: 5716080.0 | Presupuesto: 20000000 |Return of Invertion: 0.285804 | 
Título: Hang 'em High | Lanzamiento: 1968.0 | Recaudacion: 6800000.0 | Presupuesto: 1800000 |Return of Invertion: 3.7777777777777777 | 
Título: Snow Dogs | Lanzamiento: 2002.0 | Recaudacion: 0.0 | Presupuesto: 33000000 |Return of Invertion: 0.0 | 
Título: The Singing Detective | Lanzamiento: 2003.0 | Recaudacion: 0.0 | Presupuesto: 0 |Return of Invertion: 0.0 | 
Título: The Passion of the Christ | Lanzamiento: 2004.0 | Recaudacion: 611899420.0 | Presupuesto: 30000000 |Return of Invertion: 20.39664733333333 | 
Título: Paparazzi | Lanzamiento: 

In [135]:
def buscar_actor(actor_nombre):
    coincidencias = credits_df[credits_df['cast'].str.contains(actor_nombre, case=False)]
    ids_peliculas = coincidencias['id'].tolist()
    peliculas_participadas = movies_df[movies_df['id'].isin(ids_peliculas)]
    titulos_peliculas = peliculas_participadas['title'].tolist()
    return titulos_peliculas


In [139]:
actor_buscado = "Chris Pratt"  # Nombre del actor que deseas buscar
resultados = buscar_actor(actor_buscado)  # credits_df y movies_df son tus DataFrames

print(f"Películas en las que ha participado {actor_buscado}:")
for titulo in resultados:
    print(titulo)


Películas en las que ha participado Chris Pratt:
Strangers with Candy
Wanted
Bride Wars
Jennifer's Body
Take Me Home Tonight
Moneyball
What's Your Number?
The Five-Year Engagement
10 Years
Zero Dark Thirty
Movie 43
Wieners
Deep in the Valley
Delivery Man
Her
The Lego Movie
Guardians of the Galaxy
Jurassic World
Guardians of the Galaxy Vol. 2
Jem and the Holograms
Path of Destruction
The Magnificent Seven
Marvel: 75 Years, From Pulp to Pop!
Marvel Studios: Assembling a Universe
Passengers


In [ ]:
    
#3
@app.get('/franquicia/{franquicia}')
def franquicia(franquicia:str):
    # Filtrar el DataFrame original para incluir solo filas con información sobre la franquicia
    franquicia_df = df[(df['belongs_to_collection'].notna()) & (df['belongs_to_collection'].str.contains(franquicia))]
    
    # si hay nulos
    if franquicia_df.empty:
        return "No se encontro la franquicia"
    
    # Obtener la cantidad de películas, la ganancia total y la ganancia promedio
    cantidad = franquicia_df.shape[0]
    ganancia_total = franquicia_df['revenue'].sum()
    ganancia_promedio = ganancia_total / cantidad
    
    # Retornar la información en un diccionario
    return {'franquicia': franquicia, 'cantidad': cantidad, 'ganancia_total': ganancia_total, 'ganancia_promedio': ganancia_promedio}

#4
@app.get('/peliculas_pais/{pais}')
def peliculas_pais(pais:str):
    # Contar el número de películas que se hicieron en el país especificado
    num_peliculas = len(df[df['production_countries'].str.contains(pais)])
    
    # Crear un diccionario que contenga el país y el número de películas
    return {'pais': pais, 'cantidad': num_peliculas}

#5
@app.get('/productoras/{productora}')
def productoras(productora:str):    
    df_filtrado = df[df['production_companies'].str.contains(productora, na=False)]
    ganancia_total = df_filtrado['revenue'].sum()
    cantidad = len(df_filtrado)
    return {'productora': productora, 'ganancia_total': ganancia_total, 'cantidad': cantidad}

#6
@app.get('/retorno/{pelicula}')
def retorno(pelicula:str):
    # Filtro la información de la película en base a su título
    pelicula_info = df[df['title'] == pelicula].iloc[0]
    
    # Obtengo la inversión, ganancia, retorno y año de lanzamiento de la película
    inversion = int(pelicula_info['budget'])
    ganancia = int(pelicula_info['revenue'])
    retorno = int(pelicula_info['return'])
    anio = int(pelicula_info['release_year'])
    
    # Retorno la información en un diccionario
    return {'pelicula': pelicula, 'inversion': inversion, 'ganancia': ganancia, 'retorno': retorno, 'anio': anio}

# ML. MODELO DE RECOMENDACIÓN
@app.get('/recomendacion/{titulo}')
def recomendacion(titulo:str):
    # Buscar la fila correspondiente al título de la película
    idx = Df.index[Df["title"].str.lower() == titulo.lower()].tolist()
    if len(idx) == 0:
        return "Película no encontrada"
    else:
        idx = idx[0]
    
    # Calcular la similitud de la película con todas las demás películas
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Ordenar las películas según su similitud y seleccionar las 5 más similares
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:6]
    
    # Obtener los índices de las películas recomendadas
    movie_indices = [i[0] for i in sim_scores]
    
    # Devolver los títulos de las películas recomendadas
    return list(Df["title"].iloc[movie_indices])
   